# EXAMPLE 4
## Explaining the experiment
In this example, we are investigating a 256kx8 SRAM that was exposed to radiation in static tests with different bias voltages and two patterns, 0x55 and 0xAA. Several irradiations were done yielding the CSV files present in this folder. 

## Loading packages
The very first thing we must do is to load the packages required to load files (_DelimitedFiles_) as well as the LELAPE module. I suppose you have installed both. Load is done with:

In [ ]:
### Optional
push!(LOAD_PATH, "PATH_TO_FOLDER_WITH_LELAPE.jl");
#if the following cell is not successfully executed, run this one with the details of your system.

In [1]:
using DelimitedFiles, LELAPE

┌ Info: Precompiling LELAPE [top-level]
└ @ Base loading.jl:1317


## Defining variables
Previous paragraph allows us to define several variables for checking the tests:

* Word width : 8 bits
* Memory size in words: 256k is just 2^18.
* In SRAMs, it seems more likely to succeed the XOR operation.
* Tests were static. No information about cycles is necessary.

Ok, let us use this information to set these variables:

In [2]:
LA = 2^18 # Memory size in words
WordWidth = 8 # Selfexplaining.
Operation = "XOR" # Only "XOR" or "POS" are allowed.
KeepCycles = false # This is a Bool variable and only true false are accepted.

false

## Loading data
Results are stored in three different files following the required format: 
* CSV files 
* Every row is formed as WORD ADDRESS, READ VALUE, PATTERN.
Besides, the first row contains column heading (must be skipped), separators are commas and EOL character is the standard. 

We will use the _readdlm_ function provided by the _DelimitedFiles_ package to load the first CSV file and to store everything in the new variable, DATA. Finally, it is important to indicate that DATA must be an array of UInt32 numbers. 

In [36]:
DATA1 = readdlm("ExampleFRAM01.csv", ',', UInt32, '\n', skipstart=1)

9×3 Matrix{UInt32}:
 0x00000647  0x0000007f  0x000000ff
 0x00002645  0x000000f7  0x000000ff
 0x00011311  0x000000fe  0x000000ff
 0x00011313  0x000000df  0x000000ff
 0x00011331  0x000000ef  0x000000ff
 0x00011333  0x0000007f  0x000000ff
 0x00016498  0x000000df  0x000000ff
 0x0001649b  0x000000ef  0x000000ff
 0x00016c9b  0x0000007f  0x000000ff

Good!! If you have correctly proceeded, a 9x3 unsigned integer matrix is loaded. Now, let us analyze the DATA1 but we need to define in advance several variables to set the analysis.

## Looking for MBUs
This analyisis is quite simple. We will call the _CheckMBUs_ functionthat returns the MBUs present in DATA.Input arguments are the second and third columns, and the wordwidth.

This function returns two vectors. The first one indicates in position _k_ the number of bitflips observed in the _kth_ word. The second one is a vector of vectors and contains more detailed information: not only the number of bitflips per word but the position of the flipped bit (0 = LSB, WordWidth-1 = MSB). 

In [37]:
MBUSize, MBU_bit_pos = CheckMBUs(DATA1[:,2], DATA1[:,3], WordWidth)

([1, 1, 1, 1, 1, 1, 1, 1, 1], Any[[7], [3], [0], [5], [4], [7], [5], [4], [7]])

The following loop will show how many MBUs per number of flipped bits were observed:

In [41]:
for size = 1: WordWidth
    NMBUs = length(findall(MBUSize.==size))
    NMBUs != 0 ? println("$size-bit MBUs: ", NMBUs) : nothing
end

1-bit MBUs: 9


## Looking for MCUs
As modern memories are interleaved, it is not worth investigating MBUs but MCUs. Now, the system will combine addresses in all the possible pairs and operate them to create a DV set. If there were no MCUs, their characteristics are known. 

In particular, we can state that if the expected number of elements repeated _k_ times in this set is lower than a very low positive number, it is impossible to observe this number of repetitions unless the Only SBU assumption fails. We will define this threshold as 0.001 (default, 0.05). 


Although without a solid theoretical background, it seems that using pseudoaddress instead of word address provides better results.

Some experiments seem to show that if an element with very few number of 1s in binary format is too often repeated, it is indicative of the presence of MCUs. This is the Trace Rule and, in our analyisis, we want to keep all those too often repeated elements such that contain 2 ones or less in binary format.

Finally, perhaps we know that MCUs will not very large. For example, we may guess that MCUs with more than 20 bitflips are totally rejected. Therefore, to help the software and to avoid running out of memory, we will say the program _"Don't be silly and do not expect events larger than 20!!"_ If somehow this idea was wrong, we can change this value again and repeat the calculations.

In [42]:
ϵ = 0.001   # If the expected number of elements repeated k times is lower than ϵ, 
            # we can afirm that this is virtually impossible.
UsePseudoAddress = true
TraceRuleLength = 2
LargestMCUSize = 20

20

Time to test!!! We will call the function. Deppending on the set size or even if this is your first test, it will take you more or less time (Don't get up from your chair, though!!!!)

The following instruction will look for:
1. Values that pass the self-consistency test (C1_SCY)
2. Values found after inspecting MCUs derived from self-consistency-test (C1_MCU).
3. Values with less than or equal to _TraceRuleLength_ 1s in binary format that appear too often in the DV set (C1_TRC).
4. Values that, after combining in pairs the union of all the previous three sets and applying the operation and that appear too many times within the DV set (C1_SHF).

The first column of each matrix are the possible values and the second one the times it appeared.

In [43]:
C1_SCY, C1_MCU, C1_TRC, C1_SHF = DetectAnomalies_FullCheck(DATA1, WordWidth, LA, Operation, TraceRuleLength, UsePseudoAddress, KeepCycles, ϵ, LargestMCUSize)


(Matrix{UInt32}(undef, 0, 2), Matrix{UInt32}(undef, 0, 2), Matrix{UInt32}(undef, 0, 2), Matrix{UInt32}(undef, 0, 2))

All of the matrix are void!!! There is no evidence of anomalies, probably due to the fact that the number of bitflips is really low. If we check the word addresses some lines above, we may feel that there must be something wrong since all the elements are in really close logical addresses. However, nothing else can be demonstrated from a statistical analysis.

Now, you can try to test ExampleFRAM02.csv and ExampleFRAM03.csv to observe a similar behavior.

## The case of a weird set of results
ExampleFRAM04.csv is much more different than the former. We do not know what happened during the irradiation, perhaps a SEFI or something alike, but the number of bitflips was extraordinarily high. Let us analyze this set.

First of all, let us load the set.

In [47]:
DATA4 = readdlm("ExampleFRAM04.csv", ',', UInt32, '\n', skipstart=1)

2594×3 Matrix{UInt32}:
 0x00000075  0x0000000a  0x000000aa
 0x000000bd  0x00000082  0x000000aa
 0x000000d5  0x00000082  0x000000aa
 0x00000295  0x000000a2  0x000000aa
 0x00000297  0x0000008a  0x000000aa
 0x0000032c  0x000000ae  0x000000aa
 0x0000032d  0x0000000a  0x000000aa
 0x000003a5  0x00000082  0x000000aa
 0x000003d5  0x0000008a  0x000000aa
 0x000003d6  0x000000a2  0x000000aa
 0x0000048c  0x000000ae  0x000000aa
 0x0000048d  0x0000000a  0x000000aa
 0x000004f4  0x000000ae  0x000000aa
          ⋮              
 0x0003fec3  0x000000a2  0x000000aa
 0x0003fee2  0x000000e2  0x000000aa
 0x0003fefa  0x000000e2  0x000000aa
 0x0003ff41  0x0000008a  0x000000aa
 0x0003ff43  0x000000a2  0x000000aa
 0x0003ff49  0x0000008a  0x000000aa
 0x0003ff4a  0x000000a2  0x000000aa
 0x0003ff91  0x0000008a  0x000000aa
 0x0003ff92  0x000000a2  0x000000aa
 0x0003ffe8  0x000000a8  0x000000aa
 0x0003ffe9  0x0000008a  0x000000aa
 0x0003ffea  0x000000ab  0x000000aa

Now, let us check the MBUs

In [48]:
MBUSize, MBU_bit_pos = CheckMBUs(DATA4[:,2], DATA4[:,3], WordWidth)

([2, 2, 2, 1, 1, 1, 2, 2, 1, 1  …  2, 1, 1, 1, 1, 1, 1, 1, 1, 1], Any[[5, 7], [3, 5], [3, 5], [3], [5], [2], [5, 7], [3, 5], [5], [3]  …  [3, 6], [5], [3], [5], [3], [5], [3], [1], [5], [0]])

In [46]:
for size = 1: WordWidth
    NMBUs = length(findall(MBUSize.==size))
    NMBUs != 0 ? println("$size-bit MBUs: ", NMBUs) : nothing
end

1-bit MBUs: 2047
2-bit MBUs: 536
3-bit MBUs: 11


Really a large number... Are they due to the accumulation of SBUs? Let us perform some calculations...

In [2]:
NF2BIT = NF2BitMCUs(DATA4, LA, "MBU", WordWidth, WordWidth, KeepCycles)
println("We expected $NF2BIT false 2-bit MCUs in this  experiment.")

LoadError: UndefVarError: NF2BitMCUs not defined

In [26]:
println("Elements appearing more than expected and passing the Self-Consistency test:\n")
for index in 1:length(C1_SCY[:, 1])
    println("Value: 0x", string(C1_SCY[index, 1], base=16, pad = 6), " --> ", Int(C1_SCY[index, 2]),".")
end

UsePseudoAddress ? L = LA*WordWidth : L = LA

print("\nOnly up to ", MaxExpectedRepetitions(NPairs(DATA1, UsePseudoAddress, WordWidth, KeepCycles), L, Operation, ϵ)-1, " repetitions are explained by randomness.")

Elements appearing more than expected and passing the Self-Consistency test:

Value: 0x000002 --> 201.
Value: 0x000006 --> 173.
Value: 0x00000e --> 127.
Value: 0x000018 --> 120.
Value: 0x000016 --> 114.
Value: 0x00001e --> 106.
Value: 0x003f80 --> 92.
Value: 0x003200 --> 91.
Value: 0x000008 --> 90.
Value: 0x000b00 --> 90.
Value: 0x002c40 --> 89.
Value: 0x00001a --> 88.
Value: 0x000400 --> 88.
Value: 0x000d00 --> 87.
Value: 0x001100 --> 86.
Value: 0x0028c0 --> 86.
Value: 0x003e00 --> 86.

Only up to 18 repetitions are explained by randomness.

In this example, it is not worth to check the other sets since they did not yield any positive result. If you had had success, you would only have to do the following:

In [27]:
C1_All = [C1_SCY; C1_MCU; C1_TRC; C1_SHF]

17×2 Matrix{UInt32}:
 0x00000002  0x000000c9
 0x00000006  0x000000ad
 0x0000000e  0x0000007f
 0x00000018  0x00000078
 0x00000016  0x00000072
 0x0000001e  0x0000006a
 0x00003f80  0x0000005c
 0x00003200  0x0000005b
 0x00000008  0x0000005a
 0x00000b00  0x0000005a
 0x00002c40  0x00000059
 0x0000001a  0x00000058
 0x00000400  0x00000058
 0x00000d00  0x00000057
 0x00001100  0x00000056
 0x000028c0  0x00000056
 0x00003e00  0x00000056

## Grouping bitflips
Now, we have discovered those values relating pairs of pseudoaddresses. Now, let us go to group events in DATA. 

The first step consists in labeling all the pseudoaddresses and grouping their assigned indexes to a matrix containing information for the possible MCUs. It is an intermediate step and is done with the instruction _MCU_Indexes_ with the required and already defined parameters. 

In [29]:
Labeled_addresses = MCU_Indexes(DATA1, Operation, C1_All[:, 1], UsePseudoAddress, WordWidth)

72 74
152 154
233 243
318 404
331 340
338 339
346 404
353 383
357 363
359 378
360 361
370 389
373 393
382 389
399 403
537 559
543 569
626 642
662 663
731 750
968 995
974 976
1205 1207
1461 1481
1462 1477
1464 1470
1529 1530
1541 1542
1568 1631
1594 1604
1614 1618
1676 1684
1680 1740
1684 1758
1686 1750
1688 1694
1690 1718
1696 1718
1698 1716
1708 1714
1710 1730
1728 1750
1738 1762
1742 1756
1860 1976
1863 1864
1878 1879
1885 1933
1885 1941
1889 1940
1892 1935
1893 1988
1899 1935
1902 1938
1913 1914
1913 1956
1916 1962
1922 1923
1925 1931
1925 1965
1947 1988
1950 1951
1958 1972
1982 1983
2077 2079
2124 2242
2133 2142
2136 2243
2151 2217
2153 2217
2157 2251
2158 2245
2168 2202
2172 2200
2370 2411
2443 2467
2466 2477
2470 2492
2474 2480
2479 2485
2632 2633
2668 2669
2686 2740
2690 2736
2693 2705
2694 2704
2696 2704
2699 2723
2724 2735
2812 2815
2822 2828
2823 2829
2824 2830
2964 2969
2966 2971
2997 3003
3000 3006
3036 3039
3068 3125
3077 3149
3083 3113
3086 3091
3092 3132
3093 3136
3103 3

673×98 Matrix{Int64}:
    1     2     0     0     0     0  …  0  0  0  0  0  0  0  0  0  0  0  0
    3     4     0     0     0     0     0  0  0  0  0  0  0  0  0  0  0  0
    5     6     0     0     0     0     0  0  0  0  0  0  0  0  0  0  0  0
    7     8     0     0     0     0     0  0  0  0  0  0  0  0  0  0  0  0
    9    26     0     0     0     0     0  0  0  0  0  0  0  0  0  0  0  0
   10    11    27    28     0     0  …  0  0  0  0  0  0  0  0  0  0  0  0
   12    13     0     0     0     0     0  0  0  0  0  0  0  0  0  0  0  0
   14    15     0     0     0     0     0  0  0  0  0  0  0  0  0  0  0  0
   17    18     0     0     0     0     0  0  0  0  0  0  0  0  0  0  0  0
   20    21     0     0     0     0     0  0  0  0  0  0  0  0  0  0  0  0
   22    23    25    24     0     0  …  0  0  0  0  0  0  0  0  0  0  0  0
   30    31     0     0     0     0     0  0  0  0  0  0  0  0  0  0  0  0
   32    33     0     0     0     0     0  0  0  0  0  0  0  0  0  0  0  0
   

You should have got a 38x2 matrix so there are 38 2-bit MCUs.  

Now, we will classify addresses with _Classify_Addresses_in_MCU()_, using the previous matrix, _Labeled_addresses_. This function returns a vector of matrices. The first element is a matrix with pseudoaddresses related to 2-bit MCUs, and the second and last one to SBUs.

In [31]:
Events = Classify_Addresses_in_MCU(DATA1, Labeled_addresses, UsePseudoAddress, WordWidth)

98-element Vector{Any}:
 UInt32[0x001400ed 0x001400f7 … 0x00143f63 0x00143f6d]
 0×97 Matrix{UInt32}
 0×96 Matrix{UInt32}
 0×95 Matrix{UInt32}
 0×94 Matrix{UInt32}
 0×93 Matrix{UInt32}
 0×92 Matrix{UInt32}
 0×91 Matrix{UInt32}
 0×90 Matrix{UInt32}
 0×89 Matrix{UInt32}
 0×88 Matrix{UInt32}
 0×87 Matrix{UInt32}
 0×86 Matrix{UInt32}
 ⋮
 UInt32[0x001ec4e9 0x001ec4eb … 0x001eebe9 0x001eebeb]
 UInt32[0x000f85fd 0x000f85ff … 0x000fbbfd 0x000fbbff]
 UInt32[0x00088beb 0x00088bed … 0x0008bf2b 0x0008bf2d; 0x001995e1 0x001995f5 … 0x0019b2b7 0x0019b2bd; 0x001bd3ad 0x001bd3bb … 0x001be82d 0x001bec2d]
 UInt32[0x0003442d 0x0003443b … 0x00035e3b 0x00037dfb; 0x00045837 0x00045839 … 0x00047df5 0x00047df7]
 UInt32[0x0002c785 0x0002c789 … 0x0002ca93 0x0002ca97; 0x0010c2a7 0x0010c2a9 … 0x0010d3a9 0x0010cfb3; … ; 0x001bd421 0x001bd921 … 0x001bea21 0x001bec21; 0x001e04e9 0x001e04eb … 0x001e36e9 0x001e36eb]
 UInt32[0x0015026b 0x00150273 … 0x00153d6b 0x00153d73; 0x00182049 0x0018204d … 0x00182b4d 0x00182d57; 0x0

---------------------
Difficult to read, isn't it? The following instruction makes the content more readable:

In [35]:
for k = 1:length(Events) 
    NMCUs = length(Events[k][:, 1])
    if NMCUs != 0
        println("Pseudoaddresses involved in $(length(Events)-k+1)-bit MCUs ($NMCUs events):")
        for row = 1:NMCUs
            for bit = 1:length(Events)-k+1
                print("0x", string(Events[k][row, bit], base=16, pad = 6), )
            
                bit != length(Events)-k+1 ? print(", ") : print("\n")

            end
        end
        println()
    end
end

Pseudoaddresses involved in 98-bit MCUs (1 events):
0x1400ed, 0x1400f7, 0x140263, 0x14026d, 0x140279, 0x1402ad, 0x1402b5, 0x14042d, 0x140523, 0x14052d, 0x1405e3, 0x1405ed, 0x14082d, 0x140837, 0x1408e9, 0x1408ed, 0x1408f7, 0x1409ad, 0x1409bb, 0x140d23, 0x140e63, 0x140e6d, 0x140eed, 0x140fad, 0x140fb5, 0x1412bb, 0x1414ad, 0x1414b5, 0x1414ed, 0x141523, 0x14152d, 0x141623, 0x14162d, 0x1416ad, 0x1417a3, 0x1417ad, 0x14186d, 0x141875, 0x1419a3, 0x1419bb, 0x141bed, 0x141bf5, 0x141cad, 0x141cb5, 0x141da3, 0x141dad, 0x141ea3, 0x141ead, 0x14212d, 0x142135, 0x142163, 0x14216d, 0x14222d, 0x142235, 0x1424ed, 0x1424fb, 0x14262d, 0x142635, 0x1426ad, 0x1429a3, 0x1429ad, 0x142a6d, 0x142aa3, 0x142aad, 0x142bed, 0x142fed, 0x143029, 0x14302d, 0x143037, 0x1430ad, 0x1430b5, 0x14316d, 0x143175, 0x1431e3, 0x143263, 0x14326d, 0x1432e3, 0x1432ed, 0x143323, 0x14332d, 0x14346d, 0x14356d, 0x143575, 0x143777, 0x143963, 0x14397b, 0x143a29, 0x143a2d, 0x143a37, 0x143aad, 0x143be9, 0x143bed, 0x143bf7, 0x143ee3, 0x143eed

0x16899d, 0x16899f, 0x16b79d, 0x16b79f
0x169ca1, 0x16a321, 0x16b0e1, 0x16b4e1
0x16a41d, 0x16a41f, 0x16af1d, 0x16af1f
0x173bab, 0x173bad, 0x173bb1, 0x173baf
0x18100d, 0x181483, 0x18148d, 0x1838cd
0x1885a5, 0x18a065, 0x18ab65, 0x18ad65
0x18e1a5, 0x18e1a7, 0x18e1ab, 0x18e1b1
0x199423, 0x19942d, 0x199923, 0x19a723
0x199fed, 0x199fef, 0x199ff5, 0x199ff7
0x19a2a1, 0x19a2a3, 0x19a2bb, 0x19a2bd
0x19ab61, 0x19ab63, 0x19ab71, 0x19ab77
0x19aea1, 0x19aea3, 0x19aeb7, 0x19aeb9
0x19bd61, 0x19bd63, 0x19bd77, 0x19bd79
0x1bd0f3, 0x1bef6d, 0x1bef73, 0x1bef7b
0x1c26b3, 0x1c26bd, 0x1c2bab, 0x1c2bb3
0x1e0c29, 0x1e0c2b, 0x1e3e29, 0x1e3e2b
0x1e0cab, 0x1e0cb1, 0x1e0cb3, 0x1e0cb9
0x1e04f9, 0x1e19f9, 0x1e27f9, 0x1e36f9
0x1ec169, 0x1ec16b, 0x1ecc69, 0x1ecc6b
0x1fcd93, 0x1fd88d, 0x1fd893, 0x1fdc93

Pseudoaddresses involved in 3-bit MCUs (91 events):
0x018d29, 0x018d31, 0x018d33
0x025c0b, 0x025c0d, 0x02704d
0x028225, 0x02822b, 0x028235
0x0284a5, 0x0284ab, 0x0284b5
0x029e57, 0x029e59, 0x02b699
0x02d393, 0x02ec05, 0x

0x140822
0x140b2e
0x140d34
0x140d37
0x140e6e
0x1414fd
0x141636
0x1416ba
0x141723
0x14172a
0x1418fc
0x1418fd
0x1419be
0x14216e
0x1426ba
0x142a79
0x142bb9
0x142bbd
0x1431ea
0x14332e
0x14377e
0x143968
0x143bb4
0x143bb7
0x143e39
0x143e3d
0x143ee9
0x144c1b
0x14827b
0x148437
0x14843e
0x148837
0x148c1d
0x148c25
0x1492a5
0x149f37
0x149f3a
0x14b425
0x14c507
0x14e591
0x14e595
0x14e611
0x14e615
0x15022b
0x1502df
0x1504a9
0x150813
0x150839
0x151a1f
0x151da3
0x151da4
0x151da9
0x152169
0x1521df
0x15243b
0x1526eb
0x1528a3
0x1528a4
0x1528a9
0x152cb3
0x1530a2
0x1530a9
0x15326b
0x154519
0x156333
0x158525
0x1587b7
0x1591df
0x159c3b
0x15a63b
0x15b23b
0x15b33b
0x15befe
0x160523
0x16053e
0x165025
0x168099
0x1680fe
0x168261
0x16827e
0x1682b3
0x1684bb
0x168821
0x168825
0x16883e
0x1689fe
0x168cbb
0x168f7b
0x16903b
0x169521
0x16953e
0x1698b1
0x1698bd
0x169ad9
0x169bbf
0x169da1
0x169dbe
0x169de1
0x169dfe
0x16a03e
0x16a325
0x16a5fb
0x16a7a1
0x16a7bd
0x16a8bf
0x16ac22
0x16ac3f
0x16ac61
0x16ac7e
0x16b27b
0x16b33f
0

## Are there false 2-bit MCUs?
The only-SBU Model allows estimating the number of false 2-bit events. We can use an implemented function in LELAPE:

In [27]:
NF2BIT = NF2BitMCUs(DATA4, LA, Operation, length(C4_All[:,1]), WordWidth, KeepCycles, UsePseudoAddress)
println("We expected $NF2BIT false 2-bit MCUs in this  experiment.")

We expected 0.01703488826751709 false 2-bit MCUs in this  experiment.


## Analysis completed
You can find in this folder other actual data got from the same memory. Analyze them and, if you wish, add aditional cells combining the anomalous DV values and try to do the most accurate example.